In [1]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(5)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

directory_out = './file/3.part'

if os.path.exists(directory_out):
    shutil.rmtree(directory_out)
os.makedirs(directory_out)

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

# = = = = = = = = = = = = = = =

directory_in = './file/2.url'

list_file = [file for file in sorted(os.listdir(directory_in)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_in}/{file}')]
for file in list_file:

    # = = = = = = = = = = = = = = =

    list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(f'{directory_in}/{file}',
                                                                           header=0,
                                                                           dtype=str).fillna('')

    df_input['No'] = df_input['No'].astype(int)

    count_remain = len(df_input)

    print(f'总数量：{count_remain}')
    print()

    # = = = = = = = = = = = = = = =

    def crawler(crawler_tuple):
        global list_dict_correct, list_series_error, count_remain

        crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

        # = = = = = = = = = = = = = = =

        try:
            count_retry = 0
            while True:
                count_retry += 1

                try:
                    url_request = crawler_series['Url']

                    resp = requests.get(url_request,
                                        headers=get_header(),
                                        proxies=get_proxy(),
                                        timeout=(5, 15))

                    if resp.status_code == 200:
                        soup = BeautifulSoup(resp.text, 'lxml')
                        html = etree.HTML(str(soup))

                        type_ = html.xpath('//h1/span[@id="lblPartTypeName"]/text()')[0].strip()
                        if type_:
                            break
                except KeyboardInterrupt:
                    break
                except:
                    continue

            # = = = = = = = = = = = = = = =

            dict_src, i, list_href = {}, 0, list(reversed([href.strip() for href in html.xpath('//a[@data-zoom-id="ProductPic"]/@href')]))
            for href in list_href:
                if href.startswith('http'):
                    dict_src[str(i)] = href
                    i += 1

            if not list_href:
                list_href = [href.strip() for href in html.xpath('//a[@id="ProductPic"]/@href')]
                if list_href:
                    dict_src = {'0': list_href[0]}

            # = = = = = = = = = = = = = = =

            part_number = html.xpath('//span[@id="lblProductName"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            description = html.xpath('//span[@id="lblProductDesc"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            list_text = [text.strip() for text in html.xpath('//div[@id="divAppSummary"]/p/text()')]
            while '' in list_text:
                list_text.remove('')

            application_summary = ';'.join(list_text)

            # = = = = = = = = = = = = = = =

            brand = html.xpath('//span[@id="lblBrandName"]/text()')[0].strip()

            # = = = = = = = = = = = = = = =

            value = html.xpath('//input[@id="hidfDetailApp"]/@value')
            vehicle_url = f'https://www.dormanproducts.com/{value[0].strip()}' if value else ''

            # = = = = = = = = = = = = = = =

            list_tr, dict_count = html.xpath('//section[@id="productSpec"]/div/table/tr'), {}
            for tr in list_tr:
                key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                if key in dict_count:
                    dict_count[key] += 1
                else:
                    dict_count[key] = 1

            dict_specification, dict_no = {}, {}
            for i, tr in enumerate(list_tr):
                key = tr.xpath('./th/text()')[0].strip().removesuffix(':').strip()
                if dict_count[key] == 1:
                    dict_specification[str(i)] = {key: tr.xpath('./td/text()')[0].strip()}
                else:
                    if key in dict_no:
                        dict_no[key] += 1
                    else:
                        dict_no[key] = 1
                    dict_specification[str(i)] = {f'{key}-{dict_no[key]}': tr.xpath('./td/text()')[0].strip()}

            # = = = = = = = = = = = = = = =

            list_text = [text.strip() for text in html.xpath('//section[@id="productOE"]/div/table/tbody/tr/th/text()')]
            oem = ';'.join(sorted(list(set(list_text)), key=list_text.index))

            # = = = = = = = = = = = = = = =

            with semaphore_correct:
                list_dict_correct.append({'No': crawler_series['No'],
                                          'Type': type_,
                                          'Part Number': part_number,
                                          'Brand': brand,
                                          'Description': description,
                                          'Application Summary': application_summary,
                                          'OEM': oem,
                                          'Picture': '',
                                          'Url': url_request,
                                          'Json_Src': json.dumps(dict_src),
                                          'Vehicle Url': vehicle_url,
                                          'Json_Specification': json.dumps(dict_specification)})

            # = = = = = = = = = = = = = = =

            crawler_status = True
        except:
            crawler_series['Request_Url'] = url_request

            with semaphore_error:
                list_series_error.append(crawler_series)

        # = = = = = = = = = = = = = = =

        if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and count_remain == 1 or count_remain % 300 == 0 and count_remain != 0:
            clear_output()
        print(f'''[{crawler_status}] - [请求{count_retry}次] - [剩余{count_remain - 1}条] - [{file.removesuffix('.xlsx')}（{(list_file.index(file) + 1) / len(list_file) * 100:.2f}%）] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

        with semaphore_remain:
            count_remain -= 1

    # = = = = = = = = = = = = = = =

    pool.map(crawler, df_input.iterrows())

    print('输出ing...')
    print()
    if list_dict_correct:
        df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['Type', 'Part Number'],
                                                                 ascending=[True, True],
                                                                 ignore_index=True)
        df_correct['No'] = range(1, len(df_correct) + 1)
        df_correct.to_excel(f'''{directory_out}/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if list_series_error:
        df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                               ascending=[True],
                                                               ignore_index=True)
        df_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = [file for file in sorted(os.listdir(directory_out)) if file not in ['.DS_Store'] and os.path.isfile(f'{directory_out}/{file}')]
for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'{directory_out}/{file}',
              f'''{directory_out}/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：93

[True] - [请求1次] - [剩余92条] - [crawler_1（100.00%）] - [2024/10/16 15:36:51] - 18. https://www.dormanproducts.com/p-27241-34182.aspx?origin=keyword

[True] - [请求1次] - [剩余91条] - [crawler_1（100.00%）] - [2024/10/16 15:36:51] - 9. https://www.dormanproducts.com/p-84645-34248.aspx?origin=keyword

[True] - [请求1次] - [剩余90条] - [crawler_1（100.00%）] - [2024/10/16 15:36:51] - 6. https://www.dormanproducts.com/p-89000-34275.aspx?origin=keyword

[True] - [请求1次] - [剩余89条] - [crawler_1（100.00%）] - [2024/10/16 15:36:51] - 20. https://www.dormanproducts.com/p-19727-34150.aspx?origin=keyword

[True] - [请求1次] - [剩余88条] - [crawler_1（100.00%）] - [2024/10/16 15:36:51] - 12. https://www.dormanproducts.com/p-19729-34195.aspx?origin=keyword

[True] - [请求1次] - [剩余87条] - [crawler_1（100.00%）] - [2024/10/16 15:36:52] - 17. https://www.dormanproducts.com/p-2082-34185.aspx?origin=keyword

[True] - [请求1次] - [剩余86条] - [crawler_1（100.00%）] - [2024/10/16 15:36:52] - 16. https://www.dormanproducts.com/p-27242-34186.a

Progress: 100%|██████████████████████████████| 1/1 [00:00<00:00, 2057.04it/s]

Done ~
